# Imports

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import altair as alt

# Data Collecting

Get the data from the .csv files, clear and format a bit to get useful pandas DataFrames.  

## Different Data
* Subscribed Subreddits
* IP Logs
* Post Votes
* Comment Votes
* Comments
* Posts

In [2]:
# Data path
data_path = "./data/"

## Subscribed Subreddits

This data holds the subreddits that I am _currently_ subscribed to. It does not hold a history or any temporal data.  

Reddit subreddits doesn't include a way to distinguish them from each other by some kind of a tag or topic system. However, to better analyse them a tag system might be useful. From a list of basic tags subs can be tagged by hand. If the read data has a sub missing tags then it will automatically asks for tags from a pre-defined list of tags. After updating the tags data will be written onto the original .csv file.  

This data file also includes followed users (not subs). They are marked with a 'u_' prefix in the data. They will be filtered, but saved anyway since they might be useful in the future.

In [30]:
# Read the file, sort by name of the subreddits and reset the index after sorting
fname = "subscribed_subreddits.csv"
subreddits_df = pd.read_csv(data_path + fname).sort_values(by="subreddit").reset_index().drop(columns="index")

In [29]:
# Predefine the minimal amount of basic and useful tags
predefined_tags = tuple(sorted((
    'meme',
    'movie/show',
    'fandom',
    'game',
    'hobby',
    'information',
    'programming',
    'other',
    'user',
)))

retake_tags = False
try:
    # Check if the 'Tags' column exists at all
    sub_tags = subreddits_df["Tags"]
except KeyError:
    # Add an empty 'Tags' column
    subreddits_df["Tags"] = None
    sub_tags = subreddits_df["Tags"]
    retake_tags = True

# Check if there are any missing entry without tags
if sub_tags.isnull().sum() > 0:
    retake_tags = True

# Ask for tags
if retake_tags:
    save = False
    for idx in range(len(sub_tags)):
        sub = subreddits_df.loc[idx, "subreddit"]
        tags = subreddits_df.loc[idx, "Tags"]
        if type(tags) != str:
            tags = list()
            while True:
                tag = input(f"Enter a tag for the subreddit '{sub}', amongst {predefined_tags}\nEnter 'q' to finish.\n")
                if tag == "save":
                    subreddits_df.loc[idx, "Tags"] = ", ".join(sorted(tags))
                    save = True
                    break
                if tag == 'q':
                    if len(tags) != 0:
                        subreddits_df.loc[idx, "Tags"] = ", ".join(sorted(tags))
                        print("-----")
                        break
                    else:
                        print("You didn't enter any valid tags yet!")
                        continue
                if tag in predefined_tags and tag not in tags:
                    tags.append(tag)
                else:
                    print("You didn't enter a valid tag.")
        if save:
            break

subreddits_df.to_csv(data_path + fname, index=False)

Enter a tag for the subreddit 'ExposurePorn', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 hobby
Enter a tag for the subreddit 'ExposurePorn', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 other
Enter a tag for the subreddit 'ExposurePorn', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 q


-----


Enter a tag for the subreddit 'FATErpg', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 game
Enter a tag for the subreddit 'FATErpg', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 hobby
Enter a tag for the subreddit 'FATErpg', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 information
Enter a tag for the subreddit 'FATErpg', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 q


-----


Enter a tag for the subreddit 'GeekyaparLamers', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 other
Enter a tag for the subreddit 'GeekyaparLamers', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 q


...

-----


Enter a tag for the subreddit 'veYakinEvren', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 game
Enter a tag for the subreddit 'veYakinEvren', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 other
Enter a tag for the subreddit 'veYakinEvren', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 q


-----


Enter a tag for the subreddit 'yugioh', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 game
Enter a tag for the subreddit 'yugioh', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 hobby
Enter a tag for the subreddit 'yugioh', amongst ('fandom', 'game', 'hobby', 'information', 'meme', 'movie/show', 'other', 'programming', 'user')
Enter 'q' to finish.
 q


-----


In [85]:
followed_users_df = subreddits_df[subreddits_df["subreddit"].str.contains("u_")]
subreddits_df = subreddits_df[~subreddits_df["subreddit"].str.contains("u_")]

subreddits_df

,subreddit,Tags
0,AskScienceFiction,information
1,CodeBullet,other
2,DMAcademy,"game, hobby, information"
3,DaystromInstitute,"fandom, information, movie/show"
4,Deepspaceninememes,"fandom, meme, movie/show"
5,ElectroBOOM,"information, other"
6,ExposurePorn,"hobby, other"
7,FATErpg,"game, hobby, information"
8,GeekyaparLamers,other
9,GreekMythology,information


## IP Logs

IP logs data holds information about my logins to Reddit. It holds the date, time and the IP that I used. This data might be used on showing my active times even though it doesn't hold information on how long I have stayed active.  

The date data is in the form of "yyyy-mm-dd hh:mm:ss UTC". I will split the date and time, convert time into GMT+3, and name the months.

In [83]:
# Read the file, drop the first row that holds the registiration IP only, drop the IP colum and reset the indexing
fname = "ip_logs.csv"
login_datetime_df = pd.read_csv(data_path + fname).rename(columns={"date": "RawDate"}).drop(index=0, columns="ip").reset_index().drop(columns="index")

In [84]:
from datetime import datetime

# Add new columns
login_datetime_df[["Date", "MonthName", "Time"]] = None

for idx in range(len(login_datetime_df["RawDate"])):
    raw_date = login_datetime_df.loc[idx, "RawDate"].replace(" UTC", "")  # Get the time in UTC time

    # Convert datetime to local time zone
    local_datetime = datetime.fromisoformat(raw_date).astimezone()  
    datetime_dict = {"RawDate": raw_date + " UTC", "Date": None, "MonthName": None, "Time": None}  # New row to replace
    datetime_dict["Date"] = local_datetime.strftime("%d-%m-%Y")  # Get the date in the format "DD.MM.YYYY"
    datetime_dict["MonthName"] = local_datetime.strftime("%B")  # Get the full month name
    datetime_dict["Time"] = local_datetime.strftime("%H:%M%z")  # Get the time in format "HH:MM+HHMM"
    
    login_datetime_df.iloc[idx] = datetime_dict  # Insert the new row

login_datetime_df = login_datetime_df.drop(columns="RawDate")

login_datetime_df

,Date,MonthName,Time
0,29-06-2023,June,10:16+0300
1,29-06-2023,June,14:25+0300
2,30-06-2023,June,01:16+0300
3,30-06-2023,June,04:15+0300
4,30-06-2023,June,05:46+0300
...,...,...,...
357,06-10-2023,October,12:19+0300
358,06-10-2023,October,13:40+0300
359,06-10-2023,October,15:16+0300
360,07-10-2023,October,08:04+0300


## Post Votes

This data includes the posts that I have upvoted or downvoted. It includes and ID, the post link and the type of the vote; however, no temporal data.  

I will remove the ID, get the subreddit name from the URL, my vote and I will compare the sub to the subscribed subs data and get wheter or not I am subscribed to that subreddit currently.

In [108]:
# Read the file, rename the vote direction column drop the ID columns reset the indexing
fname = "post_votes.csv"
post_votes_df = pd.read_csv(data_path + fname).rename(columns={"direction": "Vote"}).drop(columns="id").reset_index().drop(columns="index")

In [117]:
# Add the new columns
for idx in range(len(post_votes_df["permalink"])):
    post_vote_dict = {"SubredditName": None, "IsSubscribed": None}
        
    # Get the sub name from the link
    permalink = post_votes_df.loc[idx, "permalink"]
    start_idx = permalink.find("r/") + 2
    stop_idx = permalink.find("/", start_idx)
    sub_name = permalink[start_idx:stop_idx]
    post_vote_dict["SubredditName"] = sub_name

    # Check if the sub is subscribed
    is_subscribed = (subreddits_df == sub_name).any().any()
    post_vote_dict["IsSubscribed"] = is_subscribed

    # Add the existing data to the new row dictionary
    post_vote_dict["permalink"] = permalink
    post_vote_dict["Vote"] = post_votes_df.loc[idx, "Vote"]

    # Add the new row data
    post_votes_df.iloc[idx] = post_vote_dict

# Drop the permalink column
post_votes_df = post_votes_df.drop(columns="permalink")
# Specify the Dtypes for later use
post_votes_df["IsSubscribed"] = post_votes_df["IsSubscribed"].astype(dtype="bool")
post_votes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222 entries, 0 to 1221
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Vote           1222 non-null   object
 1   SubredditName  1222 non-null   object
 2   IsSubscribed   1222 non-null   bool  
dtypes: bool(1), object(2)
memory usage: 20.4+ KB
